## CNN
Referencing data downloaded from https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia/version/2

### Installing the requirements
```python
pip install keras
pip install tensorflow
```

### Import relevant modules

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


### Config

In [2]:
dimension = 128
batch_size = 16

### Setup

In [3]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (dimension, dimension, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Fitting the model using train images

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(
    '../image_data/chest_xray/train/',
    target_size = (dimension, dimension),
    batch_size = batch_size,
    class_mode = 'binary'
)

test_set = test_datagen.flow_from_directory(
    '../image_data/chest_xray/test/',
    target_size = (dimension, dimension),
    batch_size = batch_size,
    class_mode = 'binary'
)

classifier.fit_generator(
    training_set,
    steps_per_epoch = 700,
    epochs = 10,
    validation_steps = 500,
    validation_data = test_set,
)

Found 5102 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/10
700/700 [==============================] - 726s 1s/step - loss: 0.2713 - acc: 0.8901 - val_loss: 0.5933 - val_acc: 0.7786
Epoch 2/10
700/700 [==============================] - 766s 1s/step - loss: 0.1789 - acc: 0.9280 - val_loss: 0.3068 - val_acc: 0.8836
Epoch 3/10
700/700 [==============================] - 758s 1s/step - loss: 0.1622 - acc: 0.9370 - val_loss: 0.6228 - val_acc: 0.7772
Epoch 4/10
700/700 [==============================] - 646s 923ms/step - loss: 0.1511 - acc: 0.9407 - val_loss: 0.3735 - val_acc: 0.8560
Epoch 5/10
700/700 [==============================] - 3929s 6s/step - loss: 0.1405 - acc: 0.9460 - val_loss: 0.4816 - val_acc: 0.8411
Epoch 6/10
700/700 [==============================] - 570s 814ms/step - loss: 0.1297 - acc: 0.9506 - val_loss: 0.6531 - val_acc: 0.8050
Epoch 7/10
700/700 [==============================] - 576s 823ms/step - loss: 0.1173 - acc: 0.9541 - val_loss: 0

### Making Predictions

In [5]:
import numpy
from keras.preprocessing import image

In [6]:
training_set.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

In [7]:
import os
predict_roots = ['../image_data/chest_xray/val/NORMAL/','../image_data/chest_xray/val/PNEUMONIA/']
results = []
for path in predict_roots:
    
    correct_class = path.split('/')[4]
    
    for filename in os.listdir(path):
        if '.png' in filename or '.jpeg' in filename:
            print('Processing {}'.format(filename))
            test_image = image.load_img(
                path + filename, 
                target_size = (dimension, dimension)
            )

            test_image = image.img_to_array(test_image)
            test_image = numpy.expand_dims(test_image, axis = 0)
            result = classifier.predict(test_image)

            if result[0][0] == 1:
                prediction = 'PNEUMONIA'
            else:
                prediction = 'NORMAL'
                
            if prediction == correct_class:
                results.append(True)
            else:
                results.append(False)
                
print("\nPrediction results: \n\n{}".format(results))
accuracy = results.count(True)/len(results)
print("\nAccuracy against model: {:.2%}".format(accuracy))

Processing NORMAL2-IM-0500-0001.jpeg
Processing NORMAL2-IM-1440-0001.jpeg
Processing IM-0545-0001-0002.jpeg
Processing NORMAL2-IM-1437-0001.jpeg
Processing IM-0450-0001.jpeg
Processing NORMAL2-IM-1351-0001.jpeg
Processing IM-0444-0001.jpeg
Processing IM-0391-0001.jpeg
Processing IM-0379-0001.jpeg
Processing IM-0135-0001.jpeg
Processing IM-0372-0001.jpeg
Processing IM-0707-0001.jpeg
Processing NORMAL2-IM-1431-0001.jpeg
Processing NORMAL2-IM-0555-0001-0001.jpeg
Processing NORMAL2-IM-0453-0001.jpeg
Processing NORMAL2-IM-1333-0001.jpeg
Processing IM-0742-0001.jpeg
Processing IM-0618-0001-0002.jpeg
Processing IM-0389-0001.jpeg
Processing IM-0631-0001.jpeg
Processing IM-0249-0001.jpeg
Processing IM-0280-0001.jpeg
Processing NORMAL2-IM-1436-0001.jpeg
Processing IM-0622-0001-0001.jpeg
Processing IM-0286-0001.jpeg
Processing IM-0539-0001-0002.jpeg
Processing IM-0145-0001.jpeg
Processing NORMAL2-IM-0507-0001.jpeg
Processing IM-0553-0001-0003.jpeg
Processing NORMAL2-IM-1342-0001.jpeg
Processing I

### Save Model

In [8]:
from keras.models import model_from_json
from keras.models import load_model

# Serialize model to JSON
model_json = classifier.to_json()

save_model_path = "./saved_models/pneumonia_model_700_500_adam.json"
save_weights_path = "./saved_models/pneumonia_model_700_500_adam.h5"

with open(save_model_path, "w") as json_file:
    json_file.write(model_json)

# Serialize weights to HDF5
classifier.save_weights(save_weights_path)

### Load Model

In [9]:
# Load json and create model
json_file = open(save_model_path, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# Load weights into new model
loaded_model.load_weights(save_weights_path)

### Example usage with loaded model 

In [10]:
single_image = image.load_img('/users/danielcorcoran/desktop/Screen Shot 2018-10-05 at 2.58.08 pm.png', target_size = (dimension, dimension))
single_image = image.img_to_array(single_image)
single_image = numpy.expand_dims(single_image, axis = 0)

loaded_model.predict(single_image)

FileNotFoundError: [Errno 2] No such file or directory: '/users/danielcorcoran/desktop/Screen Shot 2018-10-05 at 2.58.08 pm.png'